In [3]:
import numpy as np
import pandas as pd
import seaborn as sns

from scipy import stats
from matplotlib import pyplot as plt

import openpyxl

In [4]:
df_completo = pd.read_csv('./asylum_seekers.csv')

listaVertices = []

listaVertices = set(df_completo['Country / territory of asylum/residence'].unique().tolist() + df_completo['Origin'].unique().tolist())

print(len(listaVertices))

225


/tmp/ipykernel_12/3264878743.py:1: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_completo = pd.read_csv('./asylum_seekers.csv')


In [5]:
df_completo

,Year,Country / territory of asylum/residence,Origin,RSD procedure type / level,Tota pending start-year,of which UNHCR-assisted(start-year),Applied during year,decisions_recognized,decisions_other,Rejected,Otherwise closed,Total decisions,Total pending end-year,of which UNHCR-assisted(end-year)
0,2000,Zimbabwe,Afghanistan,G / FI,0,0,5,5,0,0,0,5.0,0,0
1,2000,South Africa,Afghanistan,G / FI,8,1,0,0,0,0,0,NaN,8,0
2,2000,Uzbekistan,Afghanistan,U / FI,265,265,2156,747,0,112,327,1186.0,1235,1235
3,2000,United States of America,Afghanistan,G / EO,196,0,225,151,0,31,68,250.0,171,0
4,2000,United States of America,Afghanistan,G / IN,193,0,218,182,0,51,40,273.0,150,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129715,2016,United States of America,Zimbabwe,G / IN,232,NaN,229,16,NaN,*,10,28,435,NaN
129716,2016,United States of America,Zimbabwe,G / EO,142,NaN,12,*,NaN,*,23,31,138,NaN
129717,2016,South Africa,Zimbabwe,G / AR,94,9,NaN,NaN,NaN,NaN,94,94,0,NaN
129718,2016,South Africa,Zimbabwe,G / FI,41238,4124,7964,73,NaN,7869,NaN,7942,41260,NaN


In [6]:
df_2009_2010 = df_completo.loc[(df_completo['Year'] == 2009) | (df_completo['Year'] == 2010)]
df_2009_2010

,Year,Country / territory of asylum/residence,Origin,RSD procedure type / level,Tota pending start-year,of which UNHCR-assisted(start-year),Applied during year,decisions_recognized,decisions_other,Rejected,Otherwise closed,Total decisions,Total pending end-year,of which UNHCR-assisted(end-year)
56860,2009,Pakistan,Afghanistan,U / FA,2535,2535,1178,380,0,122,1022,1524.0,2189,2189
56861,2009,Panama,Afghanistan,G / FI,0,0,4,0,0,0,0,NaN,4,4
56862,2009,Philippines,Afghanistan,G / FI,0,0,2,0,0,0,0,NaN,2,0
56863,2009,Papua New Guinea,Afghanistan,J / FA,1,1,2,2,0,0,1,3.0,0,0
56864,2009,Poland,Afghanistan,G / FI,1,0,14,0,3,4,4,11.0,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71919,2010,Ukraine,Zimbabwe,G / AR,2,1,1,0,0,0,0,NaN,3,3
71920,2010,United States of America,Zimbabwe,G / IN,26,0,106,50,0,11,58,119,15,0
71921,2010,United States of America,Zimbabwe,G / EO,202,0,31,52,0,48,54,154,0,0
71922,2010,South Africa,Zimbabwe,G / FI,0,0,146566,429,0,32750,0,33179,0,0


In [7]:
# Importando a biblioteca netpixi
from netpixi.integration.gt import Graph

# Criando um grafo não-dirigido
g = Graph(directed=True) 
g.add_vp('nome')
g.add_ep('aceitos')

In [8]:
for vertice in listaVertices:
    g.add_vertex(vertice)

In [9]:
for v in g.all_vertices():
    v['nome'] = str(v)

In [10]:
n = g.num_vertices()
print('O número de vértices do grafo é :', n)

O número de vértices do grafo é : 225


In [15]:
df_aresta = df_2009_2010.loc[:, ['Country / territory of asylum/residence', 'Origin', 'decisions_recognized']]
df_aresta = df_aresta.rename(columns={'Origin': 'origin', 'Country / territory of asylum/residence': 'destiny', 'decisions_recognized': 'value'})
df_aresta

,destiny,origin,value
56860,Pakistan,Afghanistan,380
56861,Panama,Afghanistan,0
56862,Philippines,Afghanistan,0
56863,Papua New Guinea,Afghanistan,2
56864,Poland,Afghanistan,0
...,...,...,...
71919,Ukraine,Zimbabwe,0
71920,United States of America,Zimbabwe,50
71921,United States of America,Zimbabwe,52
71922,South Africa,Zimbabwe,429


In [16]:
df_aresta.value = pd.to_numeric(df_aresta['value'], errors = 'coerce').fillna(0)
df_aresta

,destiny,origin,value
56860,Pakistan,Afghanistan,380
56861,Panama,Afghanistan,0
56862,Philippines,Afghanistan,0
56863,Papua New Guinea,Afghanistan,2
56864,Poland,Afghanistan,0
...,...,...,...
71919,Ukraine,Zimbabwe,0
71920,United States of America,Zimbabwe,50
71921,United States of America,Zimbabwe,52
71922,South Africa,Zimbabwe,429


In [17]:
df_aresta = df_aresta.groupby(['origin', 'destiny']).sum().reset_index()
df_aresta

,origin,destiny,value
0,Afghanistan,Argentina,0
1,Afghanistan,Armenia,0
2,Afghanistan,Australia,2132
3,Afghanistan,Austria,1171
4,Afghanistan,Azerbaijan,154
...,...,...,...
6293,Zimbabwe,Trinidad and Tobago,0
6294,Zimbabwe,Ukraine,0
6295,Zimbabwe,United Kingdom,4603
6296,Zimbabwe,United States of America,285


In [18]:
df_aresta = df_aresta[df_aresta['origin'] != df_aresta['destiny']]
df_aresta

,origin,destiny,value
0,Afghanistan,Argentina,0
1,Afghanistan,Armenia,0
2,Afghanistan,Australia,2132
3,Afghanistan,Austria,1171
4,Afghanistan,Azerbaijan,154
...,...,...,...
6293,Zimbabwe,Trinidad and Tobago,0
6294,Zimbabwe,Ukraine,0
6295,Zimbabwe,United Kingdom,4603
6296,Zimbabwe,United States of America,285


In [19]:
for index, row in df_aresta.iterrows():
    g.add_edge(row.origin, row.destiny)
    g.get_edge(row.origin, row.destiny)['aceitos'] = row.value

In [56]:
from netpixi.integration.gt import *

In [57]:
gt_save(g, '/grafo_20092010.net.gz')

In [58]:
g = gt_load('/grafo_20092010.net.gz')

In [59]:
import netpixi
m = gt_draw.sfdp_layout(g)
gt_move(g, m)
gt_save(g, 'grafo_20092010.net.gz')

In [60]:
r = netpixi.render('grafo_20092010.net.gz', infinite=True);

In [61]:
r.vertex_set_key('nome')

r.vertex_default(color=0xffffff)

# mudar a cor, a espessura e a curvatura das arestas
r.edge_default(color=0x20B2AA)

In [62]:
r.edge_scale('aceitos', 1, 10)

## Coreness

In [63]:
import netpixi
import cpnet

from graph_tool import spectral
from netpixi.integration.gt import *
from regression.integration.gt import *

In [64]:
g =  gt_load('/grafo_20092010.net.gz')

In [69]:
m = gt_draw.sfdp_layout(g)
gt_move(g, m)
gt_save(g, 'centralidadeCoreness20092010.net.gz')

In [70]:
r = netpixi.render('centralidadeCoreness20092010.net.gz')

In [73]:
matrix = spectral.adjacency(g)

# Além de cpnet.Rombach, há outras
# opções de algoritmos contínuos:
# cpnet.MINRES
# cpnet.Rossa

algorithm = cpnet.Rombach()
algorithm.detect(matrix)
c = algorithm.get_coreness()

# Depois do código acima, c é um dicionário
# em que as chaves são índices de vértices e
# valores são os atribuídos pelo algoritmo.

g.add_vp('coreness')
for i, coreness in c.items():
    v = g.get_vertex_by_index(i)
    v['coreness'] = float(coreness)

# Vamos usar coreness como referência para
# o tamanho dos vértices. Estabelecemos 10
# como o menor tamanho possível e 40 como
# o maior tamanho possível. Como o coreness
# está entre 0 e 1, a conta é bem simples.

for v in g.all_vertices():
    r.vertex(v['id'], size=(10 + 40 * v['coreness']))

# Quando a execução desta célula terminar,
# veja como ficou a visualização acima.

<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()


In [74]:
r.edge_scale('aceitos', 1, 10)

r.vertex_set_key('nome')

r.vertex_scale('coreness', 5, 25)

In [75]:
df_coreness = gt_data(g).sort_values("coreness", ascending = False)
df_coreness

,nome,coreness
id,,
Canada,Canada,0.994444
United States of America,United States of America,0.988889
Switzerland,Switzerland,0.983333
United Kingdom,United Kingdom,0.977778
Norway,Norway,0.972222
...,...,...
Tuvalu,Tuvalu,0.005556
Cabo Verde,Cabo Verde,0.004167
Svalbard and Jan Mayen,Svalbard and Jan Mayen,0.002778


In [76]:
import openpyxl

df_coreness.to_excel('./coreness_2009_2010.xlsx')

<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()


## OUT e IN degree

In [77]:
df_out_degree  = df_aresta.groupby('origin').sum().reindex(listaVertices).fillna(0)
df_out_degree.sort_values("value", ascending = False)

/tmp/ipykernel_12/3734551133.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_out_degree  = df_aresta.groupby('origin').sum().reindex(listaVertices).fillna(0)


,value
origin,
Myanmar,66947.0
Eritrea,53315.0
Colombia,33917.0
Somalia,27493.0
Dem. Rep. of the Congo,25745.0
...,...
Anguilla,0.0
Guadeloupe,0.0
Montserrat,0.0


In [78]:
df_in_degree  = df_aresta.groupby('destiny').sum().reindex(listaVertices).fillna(0)
df_in_degree.sort_values("value", ascending = False)

/tmp/ipykernel_12/823782808.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_in_degree  = df_aresta.groupby('destiny').sum().reindex(listaVertices).fillna(0)


,value
destiny,
Malaysia,57574.0
United States of America,38834.0
Ecuador,27729.0
France,24370.0
Canada,23459.0
...,...
Cook Islands,0.0
Saint Vincent and the Grenadines,0.0
Anguilla,0.0


In [79]:
df_out_degree.to_excel('./fluxo_saida_2009_2010.xlsx')
df_in_degree.to_excel('./fluxo_entrada_2009_2010.xlsx')

<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()


In [89]:
df_paises_in_out = df_in_degree.reset_index()
df_paises_in_out

,destiny,value
0,Bahamas,0.0
1,Dem. Rep. of the Congo,2.0
2,Mali,40.0
3,Belize,0.0
4,Morocco,283.0
...,...,...
220,Sri Lanka,114.0
221,Bermuda,0.0
222,Iceland,9.0
223,Rep. of Korea,121.0


In [92]:
df_paises_in_out = df_in_degree.reset_index()
df_paises_in_out = df_paises_in_out.rename(columns={'destiny': 'origin', 'value' : 'value'})
df_in_out = df_paises_in_out.merge(df_out_degree, on="origin", how="left")
df_in_out

,origin,value_x,value_y
0,Bahamas,0.0,9.0
1,Dem. Rep. of the Congo,2.0,25745.0
2,Mali,40.0,345.0
3,Belize,0.0,10.0
4,Morocco,283.0,120.0
...,...,...,...
220,Sri Lanka,114.0,10155.0
221,Bermuda,0.0,0.0
222,Iceland,9.0,0.0
223,Rep. of Korea,121.0,64.0


In [95]:
df_in_out = df_in_out.rename(columns={'value_x': 'in_degree', 'value_y' : 'out_degree'})

In [96]:
df_in_out.to_excel('./fluxo_total_2009_2010.xlsx')

<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()


## OPEN